### 📂 Carregamento do Dataset Refinado
Este trecho localiza automaticamente a raiz do projeto, constrói o caminho até df_base.parquet e carrega o dataset já tratado utilizando pandas.

Também valida se o arquivo existe e exibe o shape e os tipos das colunas, garantindo que a base está correta antes de iniciar a etapa de modelagem, conforme as boas práticas do projeto


In [1]:
import pandas as pd
from pathlib import Path

# Descobre a raiz do projeto (assume que o notebook está em notbooks/)
try:
    ROOT = Path(__file__).resolve().parents[1]
except NameError:
    # __file__ não existe em notebooks; usa o cwd como base
    ROOT = Path.cwd().resolve().parent

data_path = ROOT / 'data' / 'refined' / 'df_base.parquet'

if not data_path.exists():
    raise FileNotFoundError(f"Arquivo não encontrado: {data_path}")

df = pd.read_parquet(data_path)

print("Shape do dataset:", df.shape)
print("\nTipos de dados:\n")
print(df.dtypes)


Shape do dataset: (3030, 13)

Tipos de dados:

RA               str
ANO            int64
IDADE        float64
FASE           Int64
IAA          float64
IEG          float64
IDA          float64
IAN          float64
IPS          float64
IPV          float64
NOTA_MAT     float64
NOTA_POR     float64
DEFASAGEM    float64
dtype: object


### 🎯 Construção da Variável Target (ABANDONO)

Este trecho prepara a base para modelagem e define corretamente o target ABANDONO.

Primeiro, filtramos apenas fases válidas (1 a 8) e ordenamos os dados por aluno (RA) e ano, permitindo análise temporal. Em seguida, criamos uma flag que indica se o aluno apareceu no ano seguinte.

Como 2024 não possui ano posterior, marcamos apenas anos anteriores como observáveis para abandono.

Por fim, definimos ABANDONO = 1 quando:

 - O aluno estava em fase ativa (< 8),

 - O ano é observável (< 2024),

 - E o aluno não apareceu no ano seguinte.

Ao final, realizamos verificações básicas (sanity checks) para validar a distribuição do target antes do treinamento do modelo.


In [2]:
# ----------------------------------
# 0) Parâmetros do dataset
# ----------------------------------
ANO_MAX = 2024

# ----------------------------------
# 1) Filtra apenas fases válidas (1 a 8)
# ----------------------------------
df = df[df["FASE"].between(1, 8)].copy()
print("Shape após filtro de fases:", df.shape)

# ----------------------------------
# 2) Ordena para análise temporal
# ----------------------------------
df = df.sort_values(["RA", "ANO"])

# ----------------------------------
# 3) Cria flag de presença no ano seguinte
#    (apenas observacional)
# ----------------------------------
df["PRESENTE_ANO_SEGUINTE"] = (
    df.groupby("RA")["ANO"]
      .shift(-1)
      .eq(df["ANO"] + 1)
)

df["PRESENTE_ANO_SEGUINTE"] = (
    df["PRESENTE_ANO_SEGUINTE"]
    .fillna(False)
)

# ----------------------------------
# 4) Marca se o abandono é observável
#    (2024 não é observável)
# ----------------------------------
df["OBSERVAVEL_ABANDONO"] = (df["ANO"] < ANO_MAX).astype(int)

# ----------------------------------
# 5) Define ABANDONO corretamente
#    - fase ativa (< 8)
#    - ano observável (< 2024)
#    - não apareceu no ano seguinte
# ----------------------------------
df["ABANDONO"] = (
    (df["FASE"] < 8) &
    (df["OBSERVAVEL_ABANDONO"] == 1) &
    (~df["PRESENTE_ANO_SEGUINTE"])
).astype(int)

# ----------------------------------
# 6) Sanity checks
# ----------------------------------
print("\nDistribuição do target ABANDONO (apenas anos observáveis):")
print(
    df.loc[df["OBSERVAVEL_ABANDONO"] == 1, "ABANDONO"]
      .value_counts(normalize=True)
      .round(3)
)

print("\nAbandono por FASE (anos observáveis):")
print(
    df.loc[df["OBSERVAVEL_ABANDONO"] == 1]
      .groupby("FASE")["ABANDONO"]
      .mean()
      .round(3) * 100
)

print(df.dtypes)


Shape após filtro de fases: (2375, 13)

Distribuição do target ABANDONO (apenas anos observáveis):
ABANDONO
0    0.721
1    0.279
Name: proportion, dtype: float64

Abandono por FASE (anos observáveis):
FASE
1    23.3
2    25.4
3    36.1
4    31.2
5    33.6
6    43.1
7    29.5
8     0.0
Name: ABANDONO, dtype: float64
RA                           str
ANO                        int64
IDADE                    float64
FASE                       Int64
IAA                      float64
IEG                      float64
IDA                      float64
IAN                      float64
IPS                      float64
IPV                      float64
NOTA_MAT                 float64
NOTA_POR                 float64
DEFASAGEM                float64
PRESENTE_ANO_SEGUINTE       bool
OBSERVAVEL_ABANDONO        int64
ABANDONO                   int64
dtype: object


### 🧱 Dataset Final de Treinamento

Este bloco monta o df_train apenas com registros onde o abandono é observável (até 2023) e com fases treináveis (1 a 7). Em seguida, remove colunas que não devem entrar no treino (ID e variáveis técnicas/possível vazamento), padroniza os tipos de dados (incluindo o target ABANDONO) e reorganiza as colunas no formato “contrato” esperado pelo modelo.

Por fim, executa sanity checks para garantir que:

 - não existe 2024 no treino,

 - não existe FASE 8,

 - ABANDONO está apenas em {0,1},
   e imprime o schema e distribuições para validação.


In [ ]:
ANO_MAX = 2024

# ----------------------------------
# 1) Dataset FINAL para TREINAMENTO
#    - somente anos observáveis (<=2023)
#    - somente fases treináveis (1 a 7)
# ----------------------------------
df_train = df[
    (df["OBSERVAVEL_ABANDONO"] == 1) &
    (df["FASE"].between(1, 7))
].copy()

print("Shape após filtros (OBSERVAVEL + FASE 1–7):", df_train.shape)

# ----------------------------------
# 2) Remove colunas que não podem ir para treino (vazamento / técnico / id)
# ----------------------------------
cols_to_drop = ["RA", "PRESENTE_ANO_SEGUINTE", "OBSERVAVEL_ABANDONO"]
df_train = df_train.drop(columns=[c for c in cols_to_drop if c in df_train.columns])

# ----------------------------------
# 3) Garante dtypes (contrato)
# ----------------------------------
df_train["ANO"] = df_train["ANO"].astype("int64")

# FASE e FASE_IDEAL como inteiros nulos (se tiver NaN, mantém)
df_train["FASE"] = df_train["FASE"].astype("Int64")
#df_train["FASE_IDEAL"] = df_train["FASE_IDEAL"].astype("Int64")

# idade/defasagem: se forem conceitualmente inteiras, padroniza
df_train["IDADE"] = df_train["IDADE"].round().astype("Int64")
df_train["DEFASAGEM"] = df_train["DEFASAGEM"].round().astype("Int64")

# target
df_train["ABANDONO"] = df_train["ABANDONO"].astype("int64")

# ----------------------------------
# 4) Ordena colunas (contrato)
# ----------------------------------

cols_order = [
    "ANO", "IDADE", "FASE", "DEFASAGEM",
    "IAA", "IEG", "IDA", "IAN", "IPS", "IPV",
    "NOTA_MAT", "NOTA_POR",
    "ABANDONO"
]

# garante que todas existem (se alguma não existir, falha cedo)
missing_cols = [c for c in cols_order if c not in df_train.columns]
assert not missing_cols, f"Colunas faltando no df_train: {missing_cols}"

df_train = df_train[cols_order]

# ----------------------------------
# 5) Sanity checks
# ----------------------------------
assert df_train["ABANDONO"].isin([0, 1]).all(), "ABANDONO contém valores fora de {0,1}"
assert df_train["ANO"].max() <= (ANO_MAX - 1), "Treino contém 2024 (não observável)"
assert df_train["FASE"].max() <= 7, "Treino contém FASE 8 (não treinável para abandono)"
assert df_train["FASE"].min() >= 1, "Treino contém FASE < 1"

print("\nSchema final (processed/train):")
print(df_train.dtypes)

print("\nShape:", df_train.shape)

print("\nDistribuição do target:")
print(df_train["ABANDONO"].value_counts(normalize=True).round(3))

print("\nAbandono por FASE:")
print((df_train.groupby("FASE")["ABANDONO"].mean() * 100).round(1))

Shape após filtros (OBSERVAVEL + FASE 1–7): (1390, 16)

Schema final (processed/train):
ANO            int64
IDADE          Int64
FASE           Int64
DEFASAGEM      Int64
IAA          float64
IEG          float64
IDA          float64
IAN          float64
IPS          float64
IPV          float64
NOTA_MAT     float64
NOTA_POR     float64
ABANDONO       int64
dtype: object

Shape: (1390, 13)

Distribuição do target:
ABANDONO
0    0.708
1    0.292
Name: proportion, dtype: float64

Abandono por FASE:
FASE
1    23.3
2    25.4
3    36.1
4    31.2
5    33.6
6    43.1
7    29.5
Name: ABANDONO, dtype: float64


### 🔎 Análise de Valores Nulos

Este trecho calcula a quantidade e o percentual de valores nulos em cada coluna do df_train.

Em seguida, organiza essas informações em um DataFrame resumido (null_summary), ordenado do maior para o menor percentual de nulos.

Essa etapa é importante para identificar possíveis problemas de qualidade dos dados antes do treinamento do modelo, orientando decisões como imputação ou exclusão de variáveis.


In [6]:
# quantidade de nulos por coluna
null_count = df_train.isna().sum()

# percentual de nulos por coluna
null_pct = (df_train.isna().mean() * 100).round(2)

null_summary = (
    pd.DataFrame({
        "nulos": null_count,
        "% nulos": null_pct
    })
    .sort_values("% nulos", ascending=False)
)

print(null_summary)

           nulos  % nulos
NOTA_POR      16     1.15
NOTA_MAT      16     1.15
IDA           14     1.01
IEG           13     0.94
IPV           13     0.94
IPS            6     0.43
IAA            0     0.00
FASE           0     0.00
IDADE          0     0.00
ANO            0     0.00
DEFASAGEM      0     0.00
IAN            0     0.00
ABANDONO       0     0.00


### 📊 Nulos por Fase

Este trecho calcula o percentual de valores nulos por coluna dentro de cada FASE.

Agrupamos o dataset por FASE e medimos a proporção de NaN em cada variável, permitindo identificar se a ausência de dados está concentrada em fases específicas.

Essa análise ajuda a entender padrões estruturais de missing e evita decisões equivocadas de imputação global.


In [7]:
nulls_por_fase = (
    df_train
    .groupby("FASE")
    .apply(lambda x: x.isna().mean())
    .round(3) * 100
)

print(nulls_por_fase)

      ANO  IDADE  DEFASAGEM  IAA   IEG   IDA  IAN  IPS   IPV  NOTA_MAT  \
FASE                                                                     
1     0.0    0.0        0.0  0.0   0.0   0.0  0.0  0.0   0.0       0.0   
2     0.0    0.0        0.0  0.0   0.0   0.3  0.0  0.0   0.0       0.3   
3     0.0    0.0        0.0  0.0   0.0   0.0  0.0  0.0   0.0       0.7   
4     0.0    0.0        0.0  0.0   0.0   0.0  0.0  3.5   0.0       0.0   
5     0.0    0.0        0.0  0.0   0.0   0.0  0.0  0.0   0.0       0.0   
6     0.0    0.0        0.0  0.0   0.0   0.0  0.0  0.0   0.0       0.0   
7     0.0    0.0        0.0  0.0  29.5  29.5  0.0  0.0  29.5      29.5   

      NOTA_POR  ABANDONO  
FASE                      
1          0.0       0.0  
2          0.3       0.0  
3          0.7       0.0  
4          0.0       0.0  
5          0.0       0.0  
6          0.0       0.0  
7         29.5       0.0  


### 🩹 Imputação de Nulos por Fase

Esta função trata valores faltantes em variáveis numéricas imputando a mediana dentro de cada FASE, para respeitar diferenças naturais entre níveis de aprendizado.

Fluxo aplicado:

(Opcional) cria flags *_MISSING indicando quais valores eram nulos antes da imputação.

Imputa cada coluna pela mediana da própria FASE.

Se a FASE não tiver mediana válida (ex.: grupo todo nulo), usa a mediana global da coluna como fallback.

Se ainda restarem nulos (caso extremo), preenche com 0 para evitar falhas no treino.

Ao final, o código gera df_train_imp e valida que as colunas imputadas não ficaram com NaN, inclusive por FASE.


In [8]:
def imputar_nulos_por_fase(
    df_train,
    fase_col="FASE",
    cols_imputar=("IDA", "IEG", "IPV", "IPS", "NOTA_MAT", "NOTA_POR"),
    add_missing_flags=True,
    suffix_missing="_MISSING",
):
    """
    Imputa nulos de colunas numéricas usando a mediana por FASE.
    Também pode criar flags de missing (0/1) para cada coluna imputada.

    Regras:
      1) Se add_missing_flags=True: cria COL_MISSING = 1 se era nulo, senão 0
      2) Imputação principal: mediana por grupo (FASE)
      3) Fallback: se a mediana do grupo for NaN (ex.: grupo todo nulo),
         usa a mediana global da coluna.

    Retorna:
      df (cópia) com imputações e flags (se habilitado)
    """
    df = df_train.copy()

    # validações mínimas
    if fase_col not in df.columns:
        raise ValueError(f"Coluna '{fase_col}' não existe no dataframe.")

    # garante que cols_imputar existem
    cols_existentes = [c for c in cols_imputar if c in df.columns]
    cols_faltantes = [c for c in cols_imputar if c not in df.columns]
    if cols_faltantes:
        # não quebra; apenas ignora as faltantes (útil em diferentes versões do dataset)
        pass

    # cria flags de missing antes de imputar
    if add_missing_flags:
        for col in cols_existentes:
            df[f"{col}{suffix_missing}"] = df[col].isna().astype(int)

    # imputação por mediana da fase, com fallback global
    for col in cols_existentes:
        # mediana global (fallback)
        global_med = df[col].median()

        # mediana por fase
        med_por_fase = df.groupby(fase_col)[col].median()

        # função que resolve mediana do grupo com fallback global
        def _fill_group(s):
            med = med_por_fase.get(s.name)
            if med != med:  # NaN check sem numpy
                med = global_med
            return s.fillna(med)

        df[col] = df.groupby(fase_col, group_keys=False)[col].apply(_fill_group)

        # fallback final: se ainda sobrar NaN (coluna inteira NaN), zera
        if df[col].isna().any():
            df[col] = df[col].fillna(0)

    return df

df_train_imp = imputar_nulos_por_fase(df_train)

# checar nulos nas colunas imputadas
cols = ["IDA", "IEG", "IPV", "IPS", "NOTA_MAT", "NOTA_POR"]
df_train_imp[cols].isna().sum()

df_train_imp.groupby("FASE")[cols].apply(lambda x: x.isna().mean() * 100)

,IDA,IEG,IPV,IPS,NOTA_MAT,NOTA_POR
FASE,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0


### 📋 Inspeção Final do Dataset

O comando df_train_imp.info() exibe um resumo estrutural do dataset após a imputação.

Essa verificação confirma que não há mais valores faltantes nas colunas tratadas e que os tipos estão corretos antes de iniciar o treinamento do modelo.


In [9]:
df_train_imp.info()

<class 'pandas.DataFrame'>
Index: 1390 entries, 0 to 3015
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ANO               1390 non-null   int64  
 1   IDADE             1390 non-null   Int64  
 2   FASE              1390 non-null   Int64  
 3   DEFASAGEM         1390 non-null   Int64  
 4   IAA               1390 non-null   float64
 5   IEG               1390 non-null   float64
 6   IDA               1390 non-null   float64
 7   IAN               1390 non-null   float64
 8   IPS               1390 non-null   float64
 9   IPV               1390 non-null   float64
 10  NOTA_MAT          1390 non-null   float64
 11  NOTA_POR          1390 non-null   float64
 12  ABANDONO          1390 non-null   int64  
 13  IDA_MISSING       1390 non-null   int64  
 14  IEG_MISSING       1390 non-null   int64  
 15  IPV_MISSING       1390 non-null   int64  
 16  IPS_MISSING       1390 non-null   int64  
 17  NOTA_MAT_MI

### 📊 Ajuste de Tipos e Correlação com o Target

Primeiro, convertemos IDADE, FASE e DEFASAGEM para int64, garantindo consistência numérica no dataset.

Em seguida, calculamos a correlação das variáveis numéricas com o target ABANDONO, ordenando do maior para o menor valor.

Essa análise ajuda a identificar quais features possuem maior associação linear com o abandono, servindo como uma verificação exploratória antes da modelagem.


In [ ]:
int_cols = ["IDADE", "FASE", "DEFASAGEM"]
df_train_imp[int_cols] = df_train_imp[int_cols].astype("int64")

df_train_imp.corr(numeric_only=True)["ABANDONO"].sort_values(ascending=False)

ABANDONO            1.000000
IDADE               0.150355
FASE                0.091992
IPS_MISSING         0.030105
IPS                 0.011176
NOTA_POR_MISSING   -0.024820
NOTA_MAT_MISSING   -0.024820
IDA_MISSING        -0.048947
IAA                -0.061138
IPV_MISSING        -0.062412
IEG_MISSING        -0.062412
ANO                -0.070615
IAN                -0.113902
DEFASAGEM          -0.150276
NOTA_MAT           -0.200993
NOTA_POR           -0.239882
IDA                -0.255344
IPV                -0.279362
IEG                -0.342927
Name: ABANDONO, dtype: float64

### 💾 Salvando o Dataset Processado

Este trecho salva o dataset final de treinamento (df_train_imp) no formato Parquet, dentro da pasta data/processed.

O arquivo é exportado sem índice, utilizando o engine pyarrow e compressão snappy, garantindo eficiência de armazenamento e leitura.

Esse passo consolida a base pronta para ser utilizada na etapa de treinamento do modelo.


In [12]:
output_path = '../data/processed/df_trein.parquet'
df_train_imp.to_parquet(output_path, index=False, engine='pyarrow', compression='snappy')
print(f'Arquivo Parquet salvo em {output_path}')

Arquivo Parquet salvo em ../data/processed/df_trein.parquet
